# Testing the scenario.py: Node, virtual sensors, tasks and mappings

In [1]:
import sys
sys.path.append('../')

## 1. Define Airsim virtual sensors.
## 2. Define Nodes
## 3. Define infranodes
## 4. Define links between nodes/virtual sensors and infranodes
## 5. Define Tasks.
## 6. Define Dataflow between Tasks
## 7. Define Node mappings. (Node to device mappings)
## 8. Define Task mapping. (Task to Node mappings)

### Importing heliot scenario definition file

In [2]:
from scenario import *

myScenario = scenario(name='NESL_Simple_Scenario')

## 1. Airsim virtual sensor
We need two files: 
- Runfile in windows machine which points to the precompiled environment. **This is part of the testbed definition.** Every precomplied environment has a name and the path to run the environment. 
- Drone controller file of the drone which defines the virtual trajectory. Drone controller file will be run on Airsim machine by heliot to transfer the data (images)

In [4]:
virtual_drone_node = airsimSensor(type='_drone', id = 'Surveillance_drone')
virtual_drone_node.add_attribute('_controller','file_path')

myScenario.add_airsimSensor(virtual_drone_node)

2019-02-02 16:53:58,218 - scenario - ERROR - add_airsimSensor called with wrong input


SystemExit: 

## 2. Defining Nodes
- Nodes have a type. A special type of **edge**,**server** and **container** exist which leave it to heliot to map this to the devices from the testbed.

### Nvidia Tx2 node

In [5]:
tx2_node = node(type='_nvidia_jetson_tx2', id = 'Tx2_image_classifier')

myScenario.add_node(tx2_node)

2019-02-02 16:53:59,271 - scenario - ERROR - add_node called with wrong input


SystemExit: 

### Google vision kit node

In [6]:
gvt_node = node(type='_google_vision_kit', id = 'image_sensor_1')

myScenario.add_node(gvt_node)

2019-02-02 16:54:00,766 - scenario - ERROR - add_node called with wrong input


SystemExit: 

## 3. Defining Virtual Infrastructure Nodes
- These are used to connect the nodes, virtual sensors and define network characteristics

In [7]:
virtual_switch = infranode(type='_switch', id='virtual_switch_0')

myScenario.add_infranode(virtual_switch)

2019-02-02 16:54:01,306 - scenario - ERROR - add_infranode called with wrong input


SystemExit: 

## 4.  Define links between nodes/virtual sensors and infranodes

In [8]:
link_tx2_vswitch= mininetLink(name='link_tx2_vswitch', id_1='Tx2_image_classifier',id_2='virtual_switch_0')
link_tx2_vswitch.add_attribute('_latency', '2') #latency in milli seconds

link_gvk_vswitch= mininetLink(name='link_gvk_vswitch',id_1='image_sensor_1',id_2='virtual_switch_0')
link_gvk_vswitch.add_attribute('_latency', '2') #latency in milli seconds

link_drone_vswitch= mininetLink(name='link_drone_vswitch', id_1='Surveillance_drone',id_2='virtual_switch_0')
link_drone_vswitch.add_attribute('_latency', '2') #latency in milli seconds

### Add links to scenario

In [9]:
myScenario.add_mininetLink(link_tx2_vswitch)
myScenario.add_mininetLink(link_gvk_vswitch)
myScenario.add_mininetLink(link_drone_vswitch)

2019-02-02 16:54:02,090 - scenario - ERROR - add_mininetLink called with wrong input


SystemExit: 

# 4. Define Tasks

- _nodeid is where this task will be mapped.
- In this initial version, we are providing the ids of the task.
- Later we will used the core algorithms to assign mappings

In [10]:
task_gvt_camera = taskHeliot(_taskid='task_gvt_camera',_filename='Code_to_run_task',_nodeid='image_sensor_1')
task_airsim_camera = taskHeliot(_taskid='task_airsim_camera',_filename='Code_to_run_task',_nodeid='Surveillance_drone')
task_object_inference = taskHeliot(_taskid='task_object_inference',_filename='Code_to_run_task',_nodeid='Tx2_image_classifier')

# 5. Define Dataflow between Tasks 
- Multiple outputs can be taken from the tasks.
- These outputs are used to wire the dataflow as tasks.

In [11]:
gvt_image_data = task_gvt_camera.get_output("gvt_image_data") 
drone_image_data = task_airsim_camera.get_output("drone_image_data")

task_object_inference.set_input([gvt_image_data,drone_image_data])  #It can take list of output objects

('gvt_image_data', ': is set as output for task: ', 'task_gvt_camera')
('drone_image_data', ': is set as output for task: ', 'task_airsim_camera')
('gvt_image_data', ': is set as input for task:', 'task_object_inference')
('drone_image_data', ': is set as input for task:', 'task_object_inference')


# 7. Define Node mappings. (Node to device mappings)

In [12]:
# Already done.
# for example in this case
# tx2_node = node(type='_nvidia_jetson_tx2', id = 'Tx2_image_classifier')
# _nvidia_jetson_tx2 is same as device id in the testbed

# 8. Define Task mapping. (Task to Node mappings)

In [13]:
# Already done.
# for example in this case
# task_gvt_camera = taskHeliot(..,_nodeid='image_sensor_1')
# image_sensor_1 is same as node id in the scenario

# Next Starting the Scenario Topology